<img src="images/cads-logo.png" style="height: 100px;" align=left> <img src="images/apache_spark.png" style="height: 20%;width:20%" align=right>

# Apache Spark DataFrames and Spark SQL

Apache Spark is a platform for distributed data processing, and it is particularly well-suited for dealing with massive data sets. The data sets that they do not readily fit within the memory or capacity of a single server.

Apache Spark has a modular architecture. A core platform is called Apache Spark core, and there are several modules, which run on top of the core platform.

In this notebook, we will mostly learn about DataFrames and work with Spark SQL. Apache Spark supports multiple languages, including:
- Scala
- Python
- Java
- Python
- R

Apache Spark's core data structure is the Resilient Distributed Dataset (RDD). RDD is a low-level object that lets Spark work by splitting data across multiple nodes in the cluster. However, working directly with RDDs is hard. Therefore, data scientists and data engineers prefer to use the Spark DataFrame abstraction built on top of RDDs.

We are particularly interested in a data structure called DataFrames. DataFrames are a set of data that are organized into columns and rows. The columns have names, and the rows have a schema. Therefore, in this way, they are very similar or analogous to tables in relational databases. Not only DataFrames are easier to understand, but also they are more optimized for complicated operations than RDDs.

There are a couple of different ways of working with DataFrames. One way is to use the DataFrame API, and basically, that is structured around using methods on DataFrame objects. The second way is Spark SQL that allows us to enter SQL queries which are executed on DataFrames, and those DataFrames are registered as tables.

### Setup Apache Spark on Jupyter
To start working with DataFrames, first of all, we have to create a `SparkSession` object from `SparkContext`. The `SparkContext` is a connection to the running cluster, and `SparkSession` is an interface with that connection.

In [1]:
!pip install pyspark

  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=36dd5761a9c3aded6a876f04ac22d95e329752c683306f4b2f9e211f34926ed2
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\11\17\0b\53e7d10fe66ca7647d391cdba323fcf5b2f9dfcb7ebad87aa7
Successfully built pyspark


In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.getOrCreate()

FileNotFoundError: [WinError 2] The system cannot find the file specified

The previous line of code returns an existing `SparkSession` if there's already one in the environment, or creates a new one if necessary.

In [5]:
spark

NameError: name 'spark' is not defined

### Make the data set folder accessible

In the following cells, we are going to load a file called `location_temp.csv`, which is a time-series file which contains loacations of sensors and the temperatures taken at particular periods of time. 

In [ ]:
import os
MAIN_DIRECTORY = os.getcwd()

In [ ]:
file_path =MAIN_DIRECTORY+"/Data/location_temp.csv"

## Get Started with Spark DataFrames
To create a Spark DataFrame by loading a csv file, we can use `spark.read` function as follows.

In [ ]:
df1 = spark.read.format("csv").option("header","true").load(file_path)

We can use `head(n)` method to show the heading of this data frame. `n` is the number of rows and its default value is 1.

In [ ]:
df1.head(5)

If we want to show the data in a tabular format, we can use `.show(n)` method. `n` is the number of rows and its default value is 20.

In [ ]:
df1.show()

To know the number of rows in the DataFrame, there is a useful method called `count()` that performs a count on the rows in a DataFrame.

In [ ]:
df1.count()

One of the useful methods in DataFrame API is `printSchema()` that prints out the schema in the tree format.

In [ ]:
df1.printSchema()

### Rename column names

Now, let's load another file. In the data folder, we have another file called `utilization.csv`. This file does not have a header row. If we want to use the csv file schema, Spark provides an option to infer the columns' data types automatically. The following cells show how we can work with this type of csv file.

In [ ]:
file_path =MAIN_DIRECTORY+"/Data/utilization.csv"

In [ ]:
df2 = spark.read.format("csv").option("header","false").option("inferSchema","true").load(file_path)

In [ ]:
df2.count()

In [ ]:
df2.head(5)

As you can see, we have five rows, but we do not have column names. Because we did not specify a header. So Spark just created column names. Basically used a pattern `_c#`.

Spark allows us to rename the columns. By using `withColumnRenamed()` method.

In [ ]:
df2 = df2.withColumnRenamed('_c0', 'event_datetime')\
         .withColumnRenamed('_c1','server_id')\
         .withColumnRenamed('_c2','cpu_utilization')\
         .withColumnRenamed('_c3','free_memory')\
         .withColumnRenamed('_c4','session_count')

Here is the new DataFrame in the tabular format.

In [ ]:
df2.show()

In [ ]:
df2.printSchema()

Another useful method in DataFrame API is `describe()` that computes basic statistics for numeric and string columns.

This include count, mean, stddev, min, and max.

In [ ]:
df2.describe('cpu_utilization').show()

If no columns are given, this function computes statistics for all numerical or string columns.

In [ ]:
df2.describe().show()

### Load a JSON file into a DataFrame
In the following cell, we are trying to load a JSON file into a DataFrame by using `spark.read` command.

In [ ]:
file_path =MAIN_DIRECTORY+"/Data/utilization.json"

In [ ]:
df3 = spark.read.format("json").load(file_path)

In [ ]:
df3.show()

In [ ]:
df3.count()

Now, what you will notice here is we did not have to change column names.That is because in JSON, you specify key-value pairs. For example, there was a row that has `cpu_utilization` equals to 0.77, that corresponds to the first row. This row also has a key-value pair with `free_memory` equals to 0.22 and `server_id` equals to 115.

Apache Spark provides an attribute called `columns`, to show the list of a DataFrame's columns.

In [ ]:
df3.columns

Sometimes we want to work with a subset of data. For example, we have 500000 rows of data in this DataFrame. Although they are not too many rows, it may be more than you want to work with at any particular time. And you would rather work with a sample of the data. To do that, you can use `sample` command.

In [ ]:
df3_sample = df3.sample(False, 0.1)

In [ ]:
df3_sample.count()

DataFrame API provides a method called `sort()` to sort the rows based on one or more columns.

In [ ]:
df3_sort = df3.sort('server_id').show()

If we want to sort the rows based on more that one coulmn, we can specify the list of columns and sorting order by using the following syntax.

In [ ]:
df3_sorted = df3.sort(['event_datetime','server_id'], ascending = [0,1]).show()

### Filtering using DataFrame API

Now, let's take a look at how we can use DataFrame API to filter some of the rows in DataFrames.

One of the DataFrames that we have created is `df1`, which stores location ID, and temperature measurement at a particular point and time.

In [ ]:
df1.show(5)

If we want to filter rows based on their `location_id`, we can use `filter` command. `filter(condition)` filters rows using the given condition. `filter()` method essentially allows us to specify a `WHERE` clause.

In [ ]:
df1.filter(df1['location_id']=='loc0').show()

If we want to count all the rows that are located in a specific `location_id`,we can specify the `count()` command.

In [ ]:
df1.filter(df1['location_id']=='loc0').count()

Sometimes we only need to list one or two columns; in this case, we can use `select()` method that projects a set of expressions and returns a new DataFrame. Let's take a look at how it works.

In [ ]:
df1.select('location_id','temp_celcius').show()

### Aggregation using DataFrame API

Now, let's take a look at aggregating using the DataFrame API. In the following cell we will use `groupBy` method that groups the DataFrame using the specified columns, so we can run aggregation on them.

In [ ]:
df1.groupBy('location_id').count().show()

If we want to sort the DataFrame, we can use `orderBy` that returns a new DataFrame sorted by the specified column(s).

In [ ]:
df1.orderBy('location_id').show()

To calculate the average temperature at each location, we can use `agg` operation. Let's take a look at the following example.

In [ ]:
df1.groupBy('location_id').agg({'temp_celcius':'mean'}).orderBy('location_id').show()

There are different aggregation function options, for example, if we want to have the maximum temperature in each location, we can write the following code.

In [ ]:
df1.groupBy('location_id').agg({'temp_celcius':'max'}).orderBy('location_id', ascending=False).show()

### Data Sampling

Sometimes, we may want to use sampling, particularly when we have large data sets, and we are doing kind of an exploratory analysis. We want to get kind of an understanding at a high level of what the data is like. Sampling can be beneficial for doing quick operations. Now, let's see how we can take a sample, or a subset of that, but randomly. In PySpark, `sample()` method returns a sampled subset of this DataFrame, and it usually takes two parameters, `fraction` that specifies the fraction of rows to generate, range [0.0, 1.0]. The second parameter is `withReplacement`, which is a boolean parameter. Usually, we assign `false` to it, in this case, what that means is each time we pull a row out of our sampling, we don't put it back in, so we will never get duplicates, we will always get unique values. 

In [ ]:
df1_sample1 = df1.sample(fraction=0.1, withReplacement=False)

In [ ]:
df1_sample1.count()

Now, let's run some simple descriptive statistics on our sample. 

In [ ]:
df1_sample1.groupBy('location_id').agg({'temp_celcius':'mean'}).orderBy('location_id').show(10)

Now, let's compare these results to results of the original data set, the DataFrame `df1`, which has 500000 rows.

In [ ]:
df1.groupBy('location_id').agg({'temp_celcius':'mean'}).orderBy('location_id').show(10)

As you can see, when we did the sampling and took 10% when we took the average of location zero, we got something that was about 29.4, but the actual is approximately 29.18. Therefore, we can see by sampling, we get very close to what the average is for the actual population. One of the things to consider is the size of the sample that we are drawing.

### Save Data from DataFrame

Sometimes after we have been working with DataFrames and creating new DataFrames and running calculations and doing sampling, we might want to save our results out. To do this, we can use `write` object and specify the `csv()` method within that, and then specify a name or what we'd like to save. It saves the DataFrame to disk using the csv format.

In [ ]:
df1.write.csv('df1.csv')

Now, let's take a look at the current directory

In [ ]:
! ls df1.csv # in windows, you would use dir command

Now, what you will notice here is that `df1.csv` is not a single file. It is a directory. And what is in that directory is four different files with `csv` extensions, and that is because of the way Apache Spark works internally. Spark can break up DataFrames into partition subsets, and in this case, there were four partitions. Each partition has its own file. There is also a `_SUCCESS` flag that was written out. Now, let's list the contents of one of these files. 

In [ ]:
! head df1.csv/part-00000-5cb7c640-19b1-48b7-b3e5-ce37cca11d95-c000.csv

 # for Windows, use more command 

To write the DataFrame in JSON format, you can use the following code.

In [ ]:
df1.write.json('df1.json')

In [ ]:
!ls df1.json

In [ ]:
! head df1.json/part-00000-e3666b64-84b9-4168-8c6c-0ea6c727660c-c000.json

## Querying DataFrames with SQL

Up to now, we've been using the Spark DataFrame API to work with DataFrames. Now, we're going to switch gears and we're going to work with SQL. In particular, we're going to use Spark SQL for working with DataFrames.

In this part, we will use `utilization.json` that includes cpu utilization, the amount of free memory at a particular point in time, and then the number of sessions that are currently connected to the server at the particular point in time.

In [ ]:
file_path =MAIN_DIRECTORY+"/Data/utilization.json"
df_util = spark.read.format('json').load(file_path)

In [ ]:
file_path =MAIN_DIRECTORY+"/Data/utilization.csv"
df_util = spark.read.format('csv').option('inferSchema', 'true').load(file_path)

In [ ]:
df_util.show(10)

In [ ]:
df_util = df_util.withColumnRenamed('_c0', 'event_datetime')\
                .withColumnRenamed('_c1', 'server_id')\
                .withColumnRenamed('_c2', 'cpu_utilization')\
                .withColumnRenamed('_c3', 'free_memory')\
                .withColumnRenamed('_c4', 'session_count')

In [ ]:
df_util.count()

To work with SQL in Spark, we have to create a temporary view. And to do that, we specify the DataFrame, and then we call the method `createOrReplaceTempView()` and then we should specify a name for this table. Let's do it.

In [ ]:
df_util.createOrReplaceTempView("utilization")

Now, we have the ability to query on a table called utilization. We will create that by executing a SQL command in the Spark session.

In [ ]:
df_sql = spark.sql("SELECT * FROM utilization LIMIT 10")

In [ ]:
df_sql.show(20)

If we want to project on specific columns, we can do it in the following way.

In [ ]:
spark.sql("SELECT server_id, free_memory FROM utilization LIMIT 10").show()

### Filtering DataFrames with SQL
Next, we are going to take a look at how to filter DataFrames using Spark SQL.  

In [ ]:
# Example 1
spark.sql("SELECT * FROM utilization WHERE server_id = 149 LIMIT 10").show()

In [ ]:
# Example 2
spark.sql("SELECT server_id as sid, session_count as sc \
            FROM utilization WHERE session_count >50 LIMIT 10").show()

In [ ]:
# Example 3
spark.sql("SELECT server_id, session_count FROM utilization \
           WHERE session_count >50 AND server_id = 120 \
           ORDER BY session_count DESC \
           LIMIT 10").show()

### Aggregation DataFrames with SQL

When we work with SQL in databases, we often use SQL to perform aggregations and the same holds true when working with SQL in Spark. Let's write some basic queries against the DataFrame and do a very simple aggregations.

In [ ]:
# Example 1
spark.sql("SELECT count(*) FROM utilization").show()

In [ ]:
# Example 2
spark.sql("SELECT count(*) FROM utilization WHERE session_count > 70").show()

In [ ]:
# Example 3
spark.sql("SELECT server_id, count(*) FROM utilization \
           WHERE session_count > 70 \
           GROUP BY server_id").show()

In [ ]:
# Example 4
spark.sql("SELECT server_id, count(*) FROM utilization \
           WHERE session_count > 70 \
           GROUP BY server_id \
           ORDER BY count(*) DESC").show()

In [ ]:
# Example 5
spark.sql("SELECT server_id, min(session_count), max(session_count), \
           round(avg(session_count),2) \
           FROM utilization \
           WHERE session_count > 70 \
           GROUP BY server_id \
           ORDER BY count(*) DESC").show()

### Joining DataFrames with SQL

One of the most useful features of SQL is the ability to join tables. We can join in Spark SQL as well.

First, we are going to create another temporary table based on the `server_names.csv` file.

In [ ]:
file_path =MAIN_DIRECTORY+"/Data/server_names.csv"
df_servers = spark.read.csv(file_path, header=True)

In [ ]:
df_servers.show()

In [ ]:
df_servers.createOrReplaceTempView('server_name')

Now, let's quickly do a check on `server_id` in `utilization` table.

In [ ]:
spark.sql('SELECT DISTINCT server_id FROM utilization ORDER BY server_id').show()

Now, let's see what the minimum and maximum of server_id is.

In [ ]:
spark.sql("SELECT min(server_id), max(server_id) FROM utilization").show()

Well, let's join these two tables.

In [ ]:
df_join = spark.sql("SELECT u.server_id, sn.server_name, u.session_count \
                     FROM utilization u \
                     INNER JOIN server_name sn \
                     ON sn.server_id = u.server_id")
df_join.show()   

### De-Duplicating with DataFrame API

When we're working with Data Frames, Spark provides some ways to de-duplicate data. So, let's take a look at how to do that. In this part also we will learn how we can create small data sets to work within the Jupiter Notebook session. Before doing anything, please restart the Jupyter kernel.

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [ ]:
sc =SparkContext.getOrCreate()

`sc` stands for `SparkContext`. It is a global variable that gives us access to the Spark Context. Here, what we want to do is create a DataFrame, and to do that, we will use `parallelize` method that creates a parallelized data structure. Spark automatically parallelize DataFrames. But now we are going to create this data manually, so we are specifying `parallelized` explicitly.

In [ ]:
rdd = sc.parallelize([Row(server_name='Server 101', cpu_utilization=85, session_count=80),
                      Row(server_name='Server 101', cpu_utilization=80, session_count=90),
                      Row(server_name='Server 102', cpu_utilization=85, session_count=80),
                      Row(server_name='Server 102', cpu_utilization=85, session_count=80)])

In [ ]:
spark = SparkSession(sc)

`toDF()` turns that parallelized data structure to into a DataFrame.

In [ ]:
df_dup = rdd.toDF()
df_dup.show()

Now, we are going to drop duplicates. To do that we can use `drop_duplicates()` method which returns a new DataFrame with duplicate rows removed, optionally only considering certain columns.

In [ ]:
  df_dup.drop_duplicates().show()

If we want to drop any time there is a duplicate in one of the columns, we can do that as well. Let's take a look at the following example.

In [ ]:
 df_dup.drop_duplicates(['server_name']).show() 

In [ ]:
 df_dup.drop_duplicates(['server_name', 'cpu_utilization']).show()

### Working with null values

It is not uncommon to have data missing from DataFrame. When we are working with SQL, we are used to work with nulls. When we working with DataFrames, the absence of data is indicated by an NA. So in this section, we are going to look how we can work with NAs and Nulls using DataFrames and Spark SQL.

In this section, we are importing a couple of things, we have not seen before. Let's take a look at them.

In [ ]:
from pyspark.sql.functions import lit #allows us to create a literal column for a dataframe
from pyspark.sql.types import StringType

Now, we are going to add a column and set that column's values equall to null or NA. In this case, we will use `lit()` function that is a way for us to interact with column literals in PySpark. Spark SQL functions lit() is used to add a new column by assigning a literal or constant value to Spark DataFrame. 

In [ ]:
df = rdd.toDF()
df_na = df.withColumn('na_col', lit(None).cast(StringType()))

In [ ]:
df_na.show()

Now, one of the things that we can do is globally replace all nulls or NAs with some value. And we can do that with `fillna()` function. 

In [ ]:
df_na.fillna('A').show()

Now, Let's create a DataFrame that has versions both with the nulls and with the As.

In [ ]:
df_union = df_na.fillna('A').union(df_na)

In [ ]:
df_union.show()

Now we can drop only rows with NAs in them.

In [ ]:
df_union.na.drop().show()

Well, let's see how we can do that with Spark SQL.

In [ ]:
df_union.createOrReplaceTempView('na_table')

In [ ]:
spark.sql('SELECT * FROM na_table WHERE na_col IS NULL').show()

## Exploratory Data Analysis with DataFrame API

DataFrame API provides some tools for some higher level tasks like exploratory data analysis. In this section, we are going to learn how to use DataFrame API for doing some basic EDA with the utilization DataFrame. First, let's take a look at this DataFrame.

In [ ]:
df_util.show()

In [ ]:
df_util.count() 

One of the useful methods for doing exploratory data analysis is `.describe()`. Let's see how it works.

In [ ]:
df_util.describe().show()

`.describe()` actually produces another DataFrame with summary statistics about the DataFrame. For example, in this case, we see that there are several columns; there is a summary column, followed by the name of a column in the original DataFrame. For each of those columns in the original DataFrame, we have the same statistics that are calculated.
Using `.describe()`  is an excellent way to get a high-level view of what a data set might be like.

Another statistics we often want to know, is there a correlation between two of the variables?

In [ ]:
df_util.printSchema()

In [ ]:
df_util.stat.corr('cpu_utilization', 'free_memory')

In [ ]:
df_util.stat.corr('session_count','free_memory')

Sometimes we want to know how frequent are some items, what are the most frequently occurring items?

There is a method called `.freq()` items for frequent items, which we can use with the DataFrame.

In [ ]:
df_freqItems = df_util.stat.freqItems(['server_id'])

In [ ]:
df_util.stat.freqItems(['server_id','session_count']).show()

We can create a result-set that shows some basic statistics for one of the columns by using Spark SQL. Let's do it.

In [ ]:
spark.sql("SELECT min(cpu_utilization),\
            max(cpu_utilization),stddev(cpu_utilization) FROM utilization").show()

And if we want to group the result-set by `server_id`, we can write the following query.

In [ ]:
spark.sql("SELECT server_id, min(cpu_utilization),max(cpu_utilization),stddev(cpu_utilization) \
           FROM utilization \
           GROUP BY server_id").show()

Now, we are going to calculate statistics on buckets or histograms of data. The idea is, rather than look at each server individually, Spark buckets values according to how frequently they occur in certain ranges. So if we want to know how often does a CPU utilization fall in the range of 1-10 or 11-20 or 21-30, all the way up to 90-91, we could put each of those CPU utilization measures into its bucket and count how many times a CPU utilization goes into that bucket. Let's do it.

In [ ]:
spark.sql("SELECT server_id, FLOOR(cpu_utilization*100/10) as Bucket FROM utilization").show()

So far, what we have done is we have listed for each server in what  CPU utilization bucket falls at a particular time. Now we want to see how often does a CPU utilization falls into one of those ten buckets.

In [ ]:
spark.sql("SELECT count(*), FLOOR(cpu_utilization*100/10) as Bucket \
           FROM utilization GROUP BY Bucket ORDER BY Bucket").show()

## Timeseries Analysis

In this section, we are going to work with timeseries data, and timeseries data has a set of measures and a timestamp associated with them. First, let's take a look at utilization table again.

In [ ]:
spark.sql("SELECT server_id, min(cpu_utilization), max(cpu_utilization), stddev(cpu_utilization) \
           FROM utilization \
           GROUP BY server_id").show()

Sometimes we might want to compare a value within a group. For example, we would like to compare the current CPU utilization for a server to the average CPU utilization of just that server, not the entire population.

We can do that using a windowing function, and in SQL, the windowing functions are specified using an `OVER...PARTITION BY` statement. So let's take a look at how to use that.

In [ ]:
spark.sql('SELECT event_datetime, server_id, cpu_utilization, \
           avg(cpu_utilization) OVER (PARTITION BY server_id) as avg_server_util \
           FROM Utilization').show()

Now, we have different timestamps for each server ID, different CPU utilization at those particular times, but in this piece of result-set, the average server utilization is always 0.7153 for server ID 112.

Now, we want to calculate the difference any one of these measurements of CPU utilization from the average of that server is?

In [ ]:
spark.sql('SELECT event_datetime, server_id, cpu_utilization, \
           avg(cpu_utilization) OVER (PARTITION BY server_id) as avg_server_util, \
           cpu_utilization - avg(cpu_utilization) OVER (PARTITION BY server_id) as delta_server_util \
           FROM Utilization').show()

That is one of the operations that we can do with the windowing functions. We can compare a particular value in a row to a value of some aggregate function applied to a sub-set of rows.

Another operation that we can do with windowing functions is looking around the neighborhood of a row. For example, we might want to calculate in a sliding window, look at the last three values and average them or look at the previous value, current value, next value, and average them. Let's do it.

In [ ]:
spark.sql('SELECT event_datetime, server_id, cpu_utilization, \
           avg(cpu_utilization) OVER (PARTITION BY server_id ORDER BY event_datetime \
                                       ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING) as avg_server_util \
           FROM Utilization').show()

#### Great Job!